In [1]:
from ase import Atoms
from ase.build import add_adsorbate, fcc111
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton

h = 1.85
d = 1.10

slab = fcc111('Cu', size=(4, 4, 2), vacuum=10.0)

slab.calc = EMT()
e_slab = slab.get_potential_energy()

molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])
molecule.calc = EMT()
e_N2 = molecule.get_potential_energy()

add_adsorbate(slab, molecule, h, 'ontop')
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

print('Adsorption energy:', e_slab + e_N2 - slab.get_potential_energy())

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 19:18:42       11.689927       1.0797
BFGSLineSearch:    1[  2] 19:18:42       11.670814       0.4090
BFGSLineSearch:    2[  4] 19:18:42       11.625880       0.0409
Adsorption energy: 0.32351942231763786


In [2]:
#The Atoms object is a collection of atoms. Here is how to define a N2 molecule by directly specifying the position of two nitrogen atoms:
from ase import Atoms
d = 1.10
molecule = Atoms('2N', positions=[(0., 0., 0.), (0., 0., d)])

In [3]:
molecule

Atoms(symbols='N2', pbc=False)

In [4]:
#You can also build crystals using  the lattice module which returns Atoms objects corresponding to common crystal structures. 
#Let us make a Cu (111) surface:
from ase.build import fcc111
slab = fcc111('Cu', size=(4,4,2), vacuum=10.0)

In [5]:
slab

Atoms(symbols='Cu32', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=...)

In [6]:
from ase.build import bcc110
slab2 = bcc110('Fe', size=(4,4,2), vacuum=10.0)

In [7]:
#In this overview we use the effective medium theory (EMT) calculator, as it is very fast and hence useful for getting started.
from ase.calculators.emt import EMT
slab.calc = EMT()
molecule.calc = EMT()

In [8]:
#calculate the total energies for the systems by using the get_potential_energy() method from the Atoms class:
e_slab = slab.get_potential_energy()
e_N2 = molecule.get_potential_energy()

In [9]:
e_slab

11.509056283569894

In [10]:
e_N2

0.440343573035614

In [11]:
# First add the adsorbate to the Cu slab, for example in the on-top position:
h = 1.85
add_adsorbate(slab, molecule, h, 'ontop')

In [12]:
#In order to speed up the relaxation, let us keep the Cu atoms fixed in the slab by using FixAtoms from the constraints module. 
#Only the N2 molecule is then allowed to relax to the equilibrium structure:
from ase.constraints import FixAtoms
constraint = FixAtoms(mask=[a.symbol != 'N' for a in slab])
slab.set_constraint(constraint)

In [13]:
#Now attach the QuasiNewton minimizer to the system and save the trajectory file. 
#Run the minimizer with the convergence criteria that the force on all atoms should be less than some fmax:


from ase.optimize import QuasiNewton
dyn = QuasiNewton(slab, trajectory='N2Cu.traj')
dyn.run(fmax=0.05)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 19:18:43       11.689927       1.0797
BFGSLineSearch:    1[  2] 19:18:43       11.670814       0.4090
BFGSLineSearch:    2[  4] 19:18:43       11.625880       0.0409


True

In [23]:
#Writing the atomic positions to a file is done with the write() function:
from ase.io import write
write('slab.xyz', slab)

In [24]:
from ase.io import read
slab_from_file = read('slab.xyz')

In [25]:
slab_from_file

Atoms(symbols='Cu32N2', pbc=[True, True, False], cell=[[10.210621920333747, 0.0, 0.0], [5.105310960166873, 8.842657971447272, 0.0], [0.0, 0.0, 22.08423447177455]], tags=..., calculator=SinglePointCalculator(...))